## Djikstra revision

code exploring alternatives with Djikstra algorithm

In [ ]:
#### import pandas as pd
import numpy as np
import geopandas as gpd
import networkx as nx
import osmnx as ox
from shapely.geometry import Point, LineString, Polygon
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp
import folium, itertools, os, time, warnings
from IPython.display import display, clear_output

warnings.filterwarnings("ignore")

### Directories

In [ ]:
os.getcwd()

In [ ]:
## Uncomment if filepath abovee is '/home/jovyan/work/RP-Kang2020/procedure/code'
#os.chdir('../../') 

os.getcwd()

### Load Data

In [ ]:
# street networks
if not os.path.exists("data/raw/public/Chicago_Network.graphml"):
    G = ox.graph_from_place('Chicago', network_type='drive') # pulling the drive network the first time will take a while
    ox.save_graphml(G, 'raw/public/Chicago_Network.graphml')
else:
    G = ox.load_graphml('raw/public/Chicago_Network.graphml', node_type=str)
ox.plot_graph(G)

In [ ]:
hospitals = gpd.read_file('./data/raw/public/HospitalData/Chicago_Hospital_Info.shp')

### Process Hospitals + Street Networks

In [ ]:
def network_setting(network):
    _nodes_removed = len([n for (n, deg) in network.out_degree() if deg ==0])
    network.remove_nodes_from([n for (n, deg) in network.out_degree() if deg ==0])
    for component in list(nx.strongly_connected_components(network)):
        if len(component)<10:
            for node in component:
                _nodes_removed+=1
                network.remove_node(node)
    for u, v, k, data in tqdm(G.edges(data=True, keys=True),position=0):
        if 'maxspeed' in data.keys():
            speed_type = type(data['maxspeed'])
            if (speed_type==str):
                # add in try/except blocks to catch maxspeed formats that don't fit Kang et al's cases
                try:
                    if len(data['maxspeed'].split(','))==2:
                        data['maxspeed_fix']=float(data['maxspeed'].split(',')[0])                  
                    elif data['maxspeed']=='signals':
                        data['maxspeed_fix']=35.0 # drive speed setting as 35 miles
                    else:
                        data['maxspeed_fix']=float(data['maxspeed'].split()[0])
                        # print("Warning: the value of", data['maxspeed'], "is being converted to 35.0")   
                except:
                    data['maxspeed_fix']= 35.0 #miles
            else:
                try:
                    data['maxspeed_fix']=float(data['maxspeed'][0].split()[0])
                except:
                    data['maxspeed_fix']= 35.0 #miles
        else:
            data['maxspeed_fix']= 35.0 #miles
        data['maxspeed_meters'] = data['maxspeed_fix']*26.8223 # convert mile to meter
        data['time'] = float(data['length'])/ data['maxspeed_meters']
    print("Removed {} nodes ({:2.4f}%) from the OSMNX network".format(_nodes_removed, _nodes_removed/float(network.number_of_nodes())))
    print("Number of nodes: {}".format(network.number_of_nodes()))
    print("Number of edges: {}".format(network.number_of_edges()))
    return(network)

def hospital_setting(hospitals, G):
    hospitals['nearest_osm']=None
    for i in tqdm(hospitals.index, desc="Find the nearest osm from hospitals", position=0):
        hospitals['nearest_osm'][i] = ox.get_nearest_node(G, [hospitals['Y'][i], hospitals['X'][i]], method='euclidean') # find the nearest node from hospital location
    print ('hospital setting is done')
    return(hospitals)

In [ ]:
# clean street network
G = network_setting(G)

# find nearest node for each hospital
hospitals = hospital_setting(hospitals, G)

In [ ]:
distances = [10,20,30]
weights = [1,0, 0.68, 0.22]
distance_unit = 'time'

### Current Functions (for reference)

In [ ]:
def dijkstra_cca(G, nearest_osm, distance, distance_unit = "time"):
    road_network = G.subgraph(nx.single_source_dijkstra_path_length(G, nearest_osm, distance, distance_unit))  
    nodes = [Point((data['x'], data['y'])) for node, data in road_network.nodes(data=True)]
    polygon = gpd.GeoSeries(nodes).unary_union.convex_hull ## to create convex hull
    polygon = gpd.GeoDataFrame(gpd.GeoSeries(polygon)) ## change polygon to geopandas
    polygon = polygon.rename(columns={0:'geometry'}).set_geometry('geometry')
    return polygon.copy(deep=True)

### Djikstra revision using dictionary instead of subgraph

In [ ]:
# select one hospital
selection = hospitals.loc[hospitals['nearest_osm']=='257157489']
selection

In [ ]:
nx.single_source_dijkstra_path_length(G, hospitals['nearest_osm'][1], weight=distances[0])

In [ ]:
hospitals.head()

In [ ]:
def dijkstra_cca(G, nearest_osm, distance, distance_unit = "time"):
    # creates subgraph (a list of nodes instead of writing a whole new network graph)
    road_network = G.subgraph(nx.single_source_dijkstra_path_length(G, nearest_osm, distance, distance_unit))  
    # creates an x and y point for each node
    nodes = [Point((data['x'], data['y'])) for node, data in road_network.nodes(data=True)]
    # constructs a multipart convex hull out of the subgraph of nodes
    polygon = gpd.GeoSeries(nodes).unary_union.convex_hull ## to create convex hull
    # turns the polygon into a geodatframme
    polygon = gpd.GeoDataFrame(gpd.GeoSeries(polygon)) ## change polygon to geopandas
    # renames geometry column
    polygon = polygon.rename(columns={0:'geometry'}).set_geometry('geometry')
    return polygon.copy(deep=True)

In [ ]:
road_network = nx.single_source_dijkstra_path_length(G, hospitals['nearest_osm'][0], distances[0], distance_unit)  

In [ ]:
len(road_network)

In [ ]:
len(hospitals)